<a href="https://colab.research.google.com/github/GrzegorzMeller/EventsDetection/blob/master/Wiki_Historical_Event_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We are predicting whether a given text (abstract from Wikipedia page) is about historical event or not. 

In [1]:
from google.colab import drive
drive.mount('/amd/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /amd/


In [2]:
!cp /amd/My\ Drive/wiki.csv /content/

In [3]:
from IPython.core.debugger import set_trace


import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

plt.style.use(style="seaborn")


In [4]:
wiki = pd.read_csv("/content/wiki.csv")

In [5]:
wiki.head()

,Unnamed: 0,Name,Abstract,Target
0,0,http://dbpedia.org/resource/1098,This article is about the year 1098. Year 1098...,1
1,1,http://dbpedia.org/resource/1948_Arab–Israeli_War,Arab–Israeli War redirects here. For other use...,1
2,2,http://dbpedia.org/resource/Battle_of_Britain,The Battle of Britain German: die Luftschlacht...,1
3,3,http://dbpedia.org/resource/Battle_of_Evesham,The Battle of Evesham 4 August 1265 was one of...,1
4,4,http://dbpedia.org/resource/Battle_of_Kursk,The Battle of Kursk was a Second World War eng...,1


Remove URLs and HTML

In [6]:
import re
import string

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


def remove_html(text):
    html = re.compile(r"<.*?>")
    return html.sub(r"", text)

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)    

In [7]:
wiki["Abstract"] = wiki.Abstract.map(lambda x: remove_URL(x))
wiki["Abstract"] = wiki.Abstract.map(lambda x: remove_html(x))
wiki["Abstract"] = wiki.Abstract.map(lambda x: remove_punct(x))

Remove stopwords

In [8]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))


def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
wiki["Abstract"] = wiki["Abstract"].map(remove_stopwords)

In [10]:
wiki.Abstract

0        article year 1098 year 1098 mxcviii common yea...
1        arab–israeli war redirects uses see arab–israe...
2        battle britain german die luftschlacht um engl...
3        battle evesham 4 august 1265 one two main batt...
4        battle kursk second world war engagement germa...
                               ...                        
22956    openbsd journal online newspaper dedicated cov...
22957    pagseguro online mobile paymentbased ecommerce...
22958    popspoken online culture news publication foun...
22959    radiobetacom internet radio portal allows user...
22960    rapidbuyr businesstobusiness dealoftheday ecom...
Name: Abstract, Length: 22961, dtype: object

### Basic NLP

In [11]:
from collections import Counter

# Count unique words
def counter_word(text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count

In [12]:
text = wiki.Abstract

counter = counter_word(text)

In [13]:
len(counter)

128803

In [14]:
counter

Counter({'article': 182,
         'year': 1322,
         '1098': 12,
         'mxcviii': 1,
         'common': 2552,
         'starting': 188,
         'friday': 21,
         'link': 131,
         'display': 250,
         'full': 337,
         'calendar': 108,
         'julian': 60,
         'arab–israeli': 32,
         'war': 13867,
         'redirects': 39,
         'uses': 241,
         'see': 633,
         'disambiguation': 57,
         '1948': 158,
         'first': 5566,
         'fought': 4501,
         'state': 1355,
         'israel': 314,
         'military': 2913,
         'coalition': 647,
         'arab': 336,
         'states': 3279,
         'forming': 128,
         'second': 2422,
         'stage': 366,
         'palestine': 139,
         'tension': 64,
         'conflict': 2236,
         'arabs': 117,
         'jews': 91,
         'british': 4839,
         'forces': 7011,
         'ever': 235,
         'since': 1516,
         '1917': 162,
         'balfour': 4,
       

In [42]:
num_words = len(counter)

# Max number of words in a sequence
max_length = 150

Shuffle dataset

In [16]:
from sklearn.utils import shuffle
wiki = shuffle(wiki)

In [17]:
wiki.head()

,Unnamed: 0,Name,Abstract,Target
5225,5225,http://dbpedia.org/resource/Polish–Austrian_War,polish–austrian war austropolish war part war ...,1
1930,1930,http://dbpedia.org/resource/Armenian–Parthian_War,armenian–parthian war 87 85 bce refers armies ...,1
3371,3371,http://dbpedia.org/resource/Battle_of_Hondscho...,battle hondschoote took place flanders campaig...,1
2214,2214,http://dbpedia.org/resource/Battle_of_Corydon,battle corydon minor engagement took place jul...,1
1165,1165,http://dbpedia.org/resource/Battle_of_Catirai,battle catirai january 7 1569 mapuche army toq...,1


Train / test split

In [18]:
train_size = int(wiki.shape[0] * 0.8)

train_sentences = wiki.Abstract[:train_size]
train_labels = wiki.Target[:train_size]

test_sentences = wiki.Abstract[train_size:]
test_labels = wiki.Target[train_size:]

In [19]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_sentences)

Using TensorFlow backend.


In [20]:
word_index = tokenizer.word_index

In [21]:
word_index

{'battle': 1,
 'war': 2,
 'species': 3,
 'also': 4,
 'army': 5,
 'forces': 6,
 'known': 7,
 'first': 8,
 'one': 9,
 'two': 10,
 'family': 11,
 'french': 12,
 'british': 13,
 'n': 14,
 'fought': 15,
 'found': 16,
 'north': 17,
 'south': 18,
 'new': 19,
 'film': 20,
 'genus': 21,
 'small': 22,
 'american': 23,
 'united': 24,
 'took': 25,
 'siege': 26,
 'force': 27,
 'name': 28,
 'may': 29,
 'states': 30,
 'led': 31,
 'part': 32,
 'place': 33,
 'operation': 34,
 'world': 35,
 'many': 36,
 'troops': 37,
 'large': 38,
 'city': 39,
 'military': 40,
 'years': 41,
 'campaign': 42,
 'northern': 43,
 'common': 44,
 'near': 45,
 'empire': 46,
 'three': 47,
 'german': 48,
 'long': 49,
 'general': 50,
 'second': 51,
 'including': 52,
 'river': 53,
 'several': 54,
 'victory': 55,
 'southern': 56,
 'time': 57,
 'called': 58,
 'wars': 59,
 'conflict': 60,
 'however': 61,
 'de': 62,
 'later': 63,
 'major': 64,
 'america': 65,
 'black': 66,
 'although': 67,
 'government': 68,
 'used': 69,
 'western': 70

In [22]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)

In [23]:
train_sequences[0]

[39508,
 2,
 39509,
 2,
 32,
 2,
 1782,
 397,
 2910,
 397,
 454,
 46,
 24,
 153,
 2148,
 12,
 46,
 3043,
 2,
 271,
 6,
 39510,
 1555,
 2591,
 2749,
 6,
 153,
 4089,
 15,
 454,
 46,
 29,
 128,
 46,
 903,
 964,
 271,
 37,
 4791,
 454,
 72,
 2591,
 2543,
 31236,
 1351,
 2591,
 126,
 7356,
 220,
 39511,
 545,
 52,
 10044,
 8911,
 1480,
 1343,
 2475,
 2591,
 9627,
 1783]

In [43]:
from keras.preprocessing.sequence import pad_sequences

train_padded = pad_sequences(
    train_sequences, maxlen=max_length, padding="post", truncating="post"
)

In [44]:
train_padded[0]

array([39508,     2, 39509,     2,    32,     2,  1782,   397,  2910,
         397,   454,    46,    24,   153,  2148,    12,    46,  3043,
           2,   271,     6, 39510,  1555,  2591,  2749,     6,   153,
        4089,    15,   454,    46,    29,   128,    46,   903,   964,
         271,    37,  4791,   454,    72,  2591,  2543, 31236,  1351,
        2591,   126,  7356,   220, 39511,   545,    52, 10044,  8911,
        1480,  1343,  2475,  2591,  9627,  1783,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [45]:
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(
    test_sequences, maxlen=max_length, padding="post", truncating="post"
)

In [46]:
print(wiki.iloc[0].Abstract)
print(train_sequences[0])

polish–austrian war austropolish war part war fifth coalition 1809 coalition austrian empire united kingdom napoleons french empire bavaria war polish forces napoleonallied duchy warsaw assisted forces kingdom saxony fought austrian empire may russian empire joined austria polish troops withstood austrian attack warsaw defeating raszyn abandoned warsaw order reconquer parts prepartition poland including kraków lwów forcing austrians abandon warsaw futile pursuit
[39508, 2, 39509, 2, 32, 2, 1782, 397, 2910, 397, 454, 46, 24, 153, 2148, 12, 46, 3043, 2, 271, 6, 39510, 1555, 2591, 2749, 6, 153, 4089, 15, 454, 46, 29, 128, 46, 903, 964, 271, 37, 4791, 454, 72, 2591, 2543, 31236, 1351, 2591, 126, 7356, 220, 39511, 545, 52, 10044, 8911, 1480, 1343, 2475, 2591, 9627, 1783]


Check inverse

In [47]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [48]:
def decode(text):
    return " ".join([reverse_word_index.get(i, "?") for i in text])

In [49]:
decode(train_sequences[0])

'polish–austrian war austropolish war part war fifth coalition 1809 coalition austrian empire united kingdom napoleons french empire bavaria war polish forces napoleonallied duchy warsaw assisted forces kingdom saxony fought austrian empire may russian empire joined austria polish troops withstood austrian attack warsaw defeating raszyn abandoned warsaw order reconquer parts prepartition poland including kraków lwów forcing austrians abandon warsaw futile pursuit'

In [50]:
print(f"Shape of train {train_padded.shape}")
print(f"Shape of test {test_padded.shape}")

Shape of train (18368, 150)
Shape of test (4593, 150)


In [51]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.optimizers import Adam
from keras import metrics

model = Sequential()

model.add(Embedding(num_words, 50, input_length=max_length))
model.add(LSTM(64, dropout=0.1))
model.add(Dense(1, activation="sigmoid"))


optimizer = Adam(learning_rate=3e-4)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy", metrics.Precision(), metrics.Recall()])

In [36]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 300, 50)           6440150   
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 6,469,655
Trainable params: 6,469,655
Non-trainable params: 0
_________________________________________________________________


In [52]:
history = model.fit(
    train_padded, train_labels, epochs=3, validation_data=(test_padded, test_labels),
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 18368 samples, validate on 4593 samples
Epoch 1/3
18368/18368 [==============================] - 106s 6ms/step - loss: 0.3890 - accuracy: 0.8089 - precision_4: 0.9125 - recall_4: 0.6803 - val_loss: 0.1128 - val_accuracy: 0.9800 - val_precision_4: 0.9890 - val_recall_4: 0.9693
Epoch 2/3
18368/18368 [==============================] - 105s 6ms/step - loss: 0.0983 - accuracy: 0.9802 - precision_4: 0.9906 - recall_4: 0.9693 - val_loss: 0.0880 - val_accuracy: 0.9815 - val_precision_4: 0.9926 - val_recall_4: 0.9689
Epoch 3/3
18368/18368 [==============================] - 105s 6ms/step - loss: 0.0804 - accuracy: 0.9843 - precision_4: 0.9896 - recall_4: 0.9787 - val_loss: 0.0954 - val_accuracy: 0.9806 - val_precision_4: 0.9845 - val_recall_4: 0.9752


In [53]:
model.save('mdl')

In [58]:
!zip -r mdl.zip mdl

  adding: mdl (deflated 17%)


In [59]:
!cp -r mdl.zip /amd/My\ Drive